In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import pandas_datareader.data as web
from scipy.optimize import minimize
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [5]:
ticker_list = ['SOL-USD','ETH-USD','BTC-USD']

In [9]:
start_date = dt.datetime(2011, 1, 1)
end_date = dt.datetime(2024, 12, 5)
returns = (yf.download(ticker_list,start_date-pd.offsets.BDay(1),end_date+pd.offsets.BDay(1))['Adj Close']
               .resample('W')
               .last()
               .pct_change().dropna())
returns.index=returns.index.tz_localize(None)
returns

[*********************100%***********************]  3 of 3 completed


Ticker,BTC-USD,ETH-USD,SOL-USD
Date,,,
2020-04-19,0.031320,0.127046,-0.312222
2020-04-26,0.068217,0.086461,0.047016
2020-05-03,0.158545,0.069004,0.069282
2020-05-10,-0.015851,-0.105880,-0.209704
2020-05-17,0.104416,0.098405,0.141434
...,...,...,...
2024-11-10,0.170685,0.299177,0.295675
2024-11-17,0.116456,-0.036245,0.128015
2024-11-24,0.090911,0.093638,0.064586
